# Introduction
1. This tutorial introduces *trade-based metrics* for hyperparameter optimization of FinRL models.
2. As the name implies, trade-based metrics are associated with the trade activity that FinRL captures in its actions tables. In general, a trade is represented by an entry in an actions file.
2. Such metrics include counts of winning and losing trades, total value of wins and losses and ratio of average market value of wins to losses.
1. In this tutorial, we will be tuning hyperparameters for Stable baselines3 models using Optuna.
2. The default model hyperparameters may not be adequate for your custom portfolio or custom state-space. Reinforcement learning algorithms are sensitive to hyperparameters, hence tuning is an important step.
3. Hyperparamters are tuned based on an objective, which needs to be maximized or minimized. ***In this tutorial, the ratio of average winning to losing trade value is used as the objective.*** This ratio is to be ***maximized***.
3. This tutorial incorporates a multi-stock framework based on the 30 stocks (aka tickers) in the DOW JONES Industrial Average. Trade metrics are calculated for each ticker and then aggregated.
7.**IMPORTANT**: While the DOW stocks represent a portfolio, portfolio optimization techniques, such as the classic Markowitz mean-variance model, are not applied in this analysis. Other FinRL tutorials and examples demonstrate portfolio optimization.




In [ ]:
#Installing FinRL
# Set colab status to trigger installs
clb = True

print(f'Preparing for colab: {clb}')
pkgs = ['FinRL', 'optuna', 'Ray/rllib','plotly','ipywidgets']
if clb:
    print(f'Installing packages: {pkgs}')

Preparing for colab: True
Installing packages: ['FinRL', 'optuna', 'Ray/rllib', 'plotly', 'ipywidgets']


In [ ]:
# Set Variables
## Fixed
tpm_hist = {}  # record tp metric values for trials
tp_metric = 'avgwl'  # specified trade_param_metric: ratio avg value win/loss
## Settable by User
n_trials = 5  # number of HP optimization runs
total_timesteps = 2000 # per HP optimization run
## Logging callback params
lc_threshold=1e-5
lc_patience=15
lc_trial_number=5

In [ ]:
%%capture
if clb:
    # installing packages
    !pip install yfinance==0.2.54
    !pip install pyfolio-reloaded  #original pyfolio no longer maintained
    !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git
    !pip install optuna
    !pip install -U "ray[rllib]"
    !pip install plotly
    !pip install ipywidgets
    !pip install -U kaleido   # enables saving plots to file
    !pip install optuna-integration[pytorch_lightning]
    !pip install pandas-market-calendars

In [ ]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import optuna
from pathlib import Path
from google.colab import files
%matplotlib inline
from finrl import config
from finrl import config_tickers
from optuna.integration import PyTorchLightningPruningCallback
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv as StockTradingEnv_numpy
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor
import joblib
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import ray
from pprint import pprint
import kaleido

import sys
sys.path.append("../FinRL-Library")

import itertools

import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f'Torch device: {device}')


/usr/local/lib/python3.11/dist-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/usr/local/lib/python3.11/dist-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


Torch device: cuda


Zipline was developed by Quantopian, which also created pyfolio. The latter is used in FinRL to calculate and display backtest results. Despite the unavailability of zipline, as reported above, pyfolio remains operational. See [here](https://github.com/quantopian/pyfolio/issues/654) for more information.

In [ ]:
## Connect to GPU for faster processing
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jun  8 14:40:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

## Collecting data and preprocessing

1.   Load DOW 30 prices
2.   Add technical indicators
3.   Create *processed_full*, the final data set for training and testing


To save time in multiple runs, if the processed_full file is available, it is read from a previously saved csv file.






In [ ]:
#Custom ticker list dataframe download
#TODO save df to avoid download
path_pf = '/content/processed_full.csv'
if Path(path_pf).is_file():
  print('Reading ticker data')
  df = pd.read_csv(path_pf)

else:
  print('Downloading ticker data')
  ticker_list = config_tickers.DOW_30_TICKER
  df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-10-01',
                     ticker_list = ticker_list).fetch_data()
  df.to_csv('ticker_data.csv')

Reading ticker data


In [ ]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [ ]:
def create_processed_full(processed):
  list_ticker = processed["tic"].unique().tolist()
  list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
  combination = list(itertools.product(list_date,list_ticker))

  processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
  processed_full = processed_full[processed_full['date'].isin(processed['date'])]
  processed_full = processed_full.sort_values(['date','tic'])

  processed_full = processed_full.fillna(0)
  processed_full.sort_values(['date','tic'],ignore_index=True).head(5)

  processed_full.to_csv('processed_full.csv')
  return processed_full

In [ ]:
#You can add technical indicators and turbulence factor to dataframe
#Just set the use_technical_indicator=True, use_vix=True and use_turbulence=True
def create_techind():
  fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

  processed = fe.preprocess_data(df)
  return processed

In [ ]:
#Load price and technical indicator data from file if available

path_pf = '/content/processed_full.csv'
if Path(path_pf).is_file():
  print('Reading processed_full data')
  processed_full = pd.read_csv(path_pf)

else:
  print('Creating processed_full file')
  processed=create_techind()
  processed_full=create_processed_full(processed)

Reading processed_full data


In [ ]:
def perturb_data(df, noise_level=0.01):
    """
    Apply random Gaussian noise to a single randomly chosen numerical column.

    Parameters:
    - df: pandas DataFrame
    - noise_level: float, proportion of noise to add (default is 0.01)
    """
    df_perturbed = df.copy()

    # Selecciona las columnas numéricas
    num_cols = df_perturbed.select_dtypes(include=np.number).columns

    if len(num_cols) == 0:
        raise ValueError("No numeric columns found in the DataFrame.")

    # Elige una al azar
    target_col = np.random.choice(num_cols)

    # Aplica ruido gaussiano
    noise = noise_level * df_perturbed[target_col] * np.random.randn(len(df_perturbed))
    df_perturbed[target_col] += noise

    return df_perturbed

# Generate multiple perturbed datasets
num_agents = 250  # Number of agents
processed_fulls = [perturb_data(processed_full)]


In [ ]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-05-01','2021-10-01')
print(f'Number of training samples: {len(train)}')
print(f'Number of testing samples: {len(trade)}')


Number of training samples: 81004
Number of testing samples: 10024


In [ ]:
perturbed_datasets = []
for _ in range(num_agents):
    df_perturbed = perturb_data(processed_full)  # Aplica ruido a una columna aleatoria
    train = data_split(df_perturbed, '2009-01-01', '2020-07-01')
    trade = data_split(df_perturbed, '2020-05-01', '2021-10-01')
    perturbed_datasets.append((train, trade))

# Ejemplo: revisar tamaño del primer conjunto
print(f'Agente 0 - Training samples: {len(perturbed_datasets[0][0])}')
print(f'Agente 0 - Testing  samples: {len(perturbed_datasets[0][1])}')

Agente 0 - Training samples: 81004
Agente 0 - Testing  samples: 10024


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 28, State Space: 281


In [ ]:
#Defining the environment kwargs

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "buy_cost_pct": [0.001] * stock_dimension,
    "sell_cost_pct": [0.001] * stock_dimension,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "num_stock_shares": [0] * stock_dimension,
    "reward_scaling": 1e-4

}
#Instantiate the training gym compatible environment
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
#Instantiate the training environment
# Also instantiate our training gent
env_train, _ = e_train_gym.get_sb_env()
#print(type(env_train))
agent = DRLAgent(env = env_train)

In [ ]:
#Instantiate the trading environment
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None, **env_kwargs)

In [ ]:
e_train_gym_list = []
env_train_list = []
agent_list = []
e_trade_gym_list = []

for train, trade in perturbed_datasets:
    # Crear entorno de entrenamiento
    e_train_gym = StockTradingEnv(df=train, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()
    agent = DRLAgent(env=env_train)

    # Crear entorno de evaluación
    e_trade_gym = StockTradingEnv(df=trade, turbulence_threshold=None, **env_kwargs)

    # Guardar en listas
    e_train_gym_list.append(e_train_gym)
    env_train_list.append(env_train)
    agent_list.append(agent)
    e_trade_gym_list.append(e_trade_gym)

# Trade performance code
The following code calculates trade performance metrics, which are then used as an objective for optimizing hyperparameter values.

There are several available metrics. In this tutorial, the default choice is the ratio of average value of winning to losing trades.


In [ ]:
#Main method
# Calculates Trade Performance for Objective
# Called from objective method
# Returns selected trade perf metric(s)
# Requires actions and associated prices

def calc_trade_perf_metric(df_actions,
                           df_prices_trade,
                           tp_metric,
                           dbg=False):

    df_actions_p, df_prices_p, tics = prep_data(df_actions.copy(),
                                                df_prices_trade.copy())
    # actions predicted by trained model on trade data
    df_actions_p.to_csv('df_actions.csv')


    # Confirms that actions, prices and tics are consistent
    df_actions_s, df_prices_s, tics_prtfl = \
        sync_tickers(df_actions_p.copy(),df_prices_p.copy(),tics)

    # copy to ensure that tics from portfolio remains unchanged
    tics = tics_prtfl.copy()

    # Analysis is performed on each portfolio ticker
    perf_data= collect_performance_data(df_actions_s, df_prices_s, tics)
    # profit/loss for each ticker
    pnl_all = calc_pnl_all(perf_data, tics)
    # values for trade performance metrics
    perf_results = calc_trade_perf(pnl_all)
    df = pd.DataFrame.from_dict(perf_results, orient='index')

    # calculate and return trade metric value as objective
    m = calc_trade_metric(df,tp_metric)
    print(f'Ratio Avg Win/Avg Loss: {m}')
    k = str(len(tpm_hist)+1)
    # save metric value
    tpm_hist[k] = m
    return m



In [ ]:
# Supporting methods
def calc_trade_metric(df,metric='avgwl'):
    '''# trades', '# wins', '# losses', 'wins total value', 'wins avg value',
       'losses total value', 'losses avg value'''
    # For this tutorial, the only metric available is the ratio of
    #  average values of winning to losing trades. Others are in development.

    # some test cases produce no losing trades.
    # The code below assigns a value as a multiple of the highest value during
    # previous hp optimization runs. If the first run experiences no losses,
    # a fixed value is assigned for the ratio
    tpm_mult = 1.0
    avgwl_no_losses = 25
    if metric == 'avgwl':
        if sum(df['# losses']) == 0:
          try:
            return max(tpm_hist.values())*tpm_mult
          except ValueError:
            return avgwl_no_losses
        avg_w = sum(df['wins total value'])/sum(df['# wins'])
        avg_l = sum(df['losses total value'])/sum(df['# losses'])
        m = abs(avg_w/avg_l)

    return m


def prep_data(df_actions,
              df_prices_trade):

    df=df_prices_trade[['date','close','tic']]
    df['Date'] = pd.to_datetime(df['date'])
    df = df.set_index('Date')
    # set indices on both df to datetime
    idx = pd.to_datetime(df_actions.index, infer_datetime_format=True)
    df_actions.index=idx
    tics = np.unique(df.tic)
    n_tics = len(tics)
    print(f'Number of tickers: {n_tics}')
    print(f'Tickers: {tics}')
    dategr = df.groupby('tic')
    p_d={t:dategr.get_group(t).loc[:,'close'] for t in tics}
    df_prices = pd.DataFrame.from_dict(p_d)
    df_prices.index = df_prices.index.normalize()
    return df_actions, df_prices, tics


# prepares for integrating action and price files
def link_prices_actions(df_a,
                        df_p):
    cols_a = [t + '_a' for t in df_a.columns]
    df_a.columns = cols_a
    cols_p = [t + '_p' for t in df_p.columns]
    df_p.columns = cols_p
    return df_a, df_p


def sync_tickers(df_actions,df_tickers_p,tickers):
    # Some DOW30 components may not be included in portfolio
    # passed tickers includes all DOW30 components
    # actions and ticker files may have different length indices
    if len(df_actions) != len(df_tickers_p):
      msng_dates = set(df_actions.index)^set(df_tickers_p.index)
      try:
        #assumption is prices has one additional timestamp (row)
        df_tickers_p.drop(msng_dates,inplace=True)
      except:
        df_actions.drop(msng_dates,inplace=True)
    df_actions, df_tickers_p = link_prices_actions(df_actions,df_tickers_p)
    # identify any DOW components not in portfolio
    t_not_in_a = [t for t in tickers if t + '_a' not in list(df_actions.columns)]

    # remove t_not_in_a from df_tickers_p
    drop_cols = [t + '_p' for t in t_not_in_a]
    df_tickers_p.drop(columns=drop_cols,inplace=True)

    # Tickers in portfolio
    tickers_prtfl = [c.split('_')[0] for c in df_actions.columns]
    return df_actions,df_tickers_p, tickers_prtfl

def collect_performance_data(dfa,dfp,tics, dbg=False):

    perf_data = {}
    # In current version, files columns include secondary identifier
    for t in tics:
        # actions: purchase/sale of DOW equities
        acts = dfa['_'.join([t,'a'])].values
        # ticker prices
        prices = dfp['_'.join([t,'p'])].values
        # market value of purchases/sales
        tvals_init = np.multiply(acts,prices)
        d={'actions':acts, 'prices':prices,'init_values':tvals_init}
        perf_data[t]=d

    return perf_data


def calc_pnl_all(perf_dict, tics_all):
    # calculate profit/loss for each ticker
    print(f'Calculating profit/loss for each ticker')
    pnl_all = {}
    for tic in tics_all:
        pnl_t = []
        tic_data = perf_dict[tic]
        init_values = tic_data['init_values']
        acts = tic_data['actions']
        prices = tic_data['prices']
        cs = np.cumsum(acts)
        args_s = [i + 1 for i in range(len(cs) - 1) if cs[i + 1] < cs[i]]
        # tic actions with no sales
        if not args_s:
            pnl = complete_calc_buyonly(acts, prices, init_values)
            pnl_all[tic] = pnl
            continue
        # copy acts: acts_rev will be revised based on closing/reducing init positions
        pnl_all = execute_position_sales(tic,acts,prices,args_s,pnl_all)

    return pnl_all


def complete_calc_buyonly(actions, prices, init_values):
    # calculate final pnl for each ticker assuming no sales
    fnl_price = prices[-1]
    final_values = np.multiply(fnl_price, actions)
    pnl = np.subtract(final_values, init_values)
    return pnl


def execute_position_sales(tic,acts,prices,args_s,pnl_all):
  # calculate final pnl for each ticker with sales
    pnl_t = []
    acts_rev = acts.copy()
    # location of sales transactions
    for s in args_s:  # s is scaler
        # price_s = [prices[s]]
        act_s = [acts_rev[s]]
        args_b = [i for i in range(s) if acts_rev[i] > 0]
        prcs_init_trades = prices[args_b]
        acts_init_trades = acts_rev[args_b]

        # update actions for sales
        # reduce/eliminate init values through trades
        # always start with earliest purchase that has not been closed through sale
        # selectors for purchase and sales trades
        # find earliest remaining purchase
        arg_sel = min(args_b)
        # sel_s = len(acts_trades) - 1

        # closing part/all of earliest init trade not yet closed
        # sales actions are negative
        # in this test case, abs_val of init and sales share counts are same
        # zero-out sales actions
        # market value of sale
        # max number of shares to be closed: may be less than # originally purchased
        acts_shares = min(abs(act_s.pop()), acts_rev[arg_sel])

        # mv of shares when purchased
        mv_p = abs(acts_shares * prices[arg_sel])
        # mv of sold shares
        mv_s = abs(acts_shares * prices[s])

        # calc pnl
        pnl = mv_s - mv_p
        # reduce init share count
        # close all/part of init purchase
        acts_rev[arg_sel] -= acts_shares
        acts_rev[s] += acts_shares
        # calculate pnl for trade
        # value of associated purchase

        # find earliest non-zero positive act in acts_revs
        pnl_t.append(pnl)

    pnl_op = calc_pnl_for_open_positions(acts_rev, prices)
    #pnl_op is list
    # add pnl_op results (if any) to pnl_t (both lists)
    pnl_t.extend(pnl_op)
    #print(f'Total pnl for {tic}: {np.sum(pnl_t)}')
    pnl_all[tic] = np.array(pnl_t)
    return pnl_all


def calc_pnl_for_open_positions(acts,prices):
    # identify any positive share values after accounting for sales
    pnl = []
    fp = prices[-1] # last price
    open_pos_arg = np.argwhere(acts>0)
    if len(open_pos_arg)==0:return pnl # no open positions

    mkt_vals_open = np.multiply(acts[open_pos_arg], prices[open_pos_arg])
    # mkt val at end of testing period
    # treat as trades for purposes of calculating pnl at end of testing period
    mkt_vals_final = np.multiply(fp, acts[open_pos_arg])
    pnl_a = np.subtract(mkt_vals_final, mkt_vals_open)
    #convert to list
    pnl = [i[0] for i in pnl_a.tolist()]
    #print(f'Market value of open positions at end of testing {pnl}')
    return pnl


def calc_trade_perf(pnl_d):
    # calculate trade performance metrics
    perf_results = {}
    for t,pnl in pnl_d.items():
        wins = pnl[pnl>0]  # total val
        losses = pnl[pnl<0]
        n_wins = len(wins)
        n_losses = len(losses)
        n_trades = n_wins + n_losses
        wins_val = np.sum(wins)
        losses_val = np.sum(losses)
        wins_avg = 0 if n_wins==0 else np.mean(wins)
        #print(f'{t} n_wins: {n_wins} n_losses: {n_losses}')
        losses_avg = 0 if n_losses==0 else np.mean(losses)
        d = {'# trades':n_trades,'# wins':n_wins,'# losses':n_losses,
             'wins total value':wins_val, 'wins avg value':wins_avg,
             'losses total value':losses_val, 'losses avg value':losses_avg,}
        perf_results[t] = d
    return perf_results

## Tuning hyperparameters using Optuna
1. Go to this [link](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/utils/hyperparams_opt.py), you will find all possible hyperparamters to tune for all the models.
2. For your model, grab those hyperparamters which you want to optimize and then return a dictionary of hyperparamters.
3. There is a feature in Optuna called as hyperparamters importance, you can point out those hyperparamters which are important for tuning.
4. By default Optuna use [TPESampler](https://www.youtube.com/watch?v=tdwgR1AqQ8Y) for sampling hyperparameters from the search space.

In [ ]:
def sample_ddpg_params(trial:optuna.Trial):
  # Size of the replay buffer
  buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1)
  batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])

  return {"buffer_size": buffer_size,
          "learning_rate":learning_rate,
          "batch_size":batch_size}

## Callbacks
1. The callback will terminate if the improvement margin is below certain point
2. It will terminate after certain number of trial_number are reached, not before that
3. It will hold its patience to reach the threshold

In [ ]:
class LoggingCallback:
    def __init__(self,threshold,trial_number,patience):
      '''
      threshold:int tolerance for increase in objective
      trial_number: int Prune after minimum number of trials
      patience: int patience for the threshold
      '''
      self.threshold = threshold
      self.trial_number  = trial_number
      self.patience = patience
      print(f'Callback threshold {self.threshold}, \
            trial_number {self.trial_number}, \
            patience {self.patience}')
      self.cb_list = [] #Trials list for which threshold is reached
    def __call__(self,study:optuna.study, frozen_trial:optuna.Trial):
      #Setting the best value in the current trial
      study.set_user_attr("previous_best_value", study.best_value)

      #Checking if the minimum number of trials have pass
      if frozen_trial.number >self.trial_number:
          previous_best_value = study.user_attrs.get("previous_best_value",None)
          #Checking if the previous and current objective values have the same sign
          if previous_best_value * study.best_value >=0:
              #Checking for the threshold condition
              if abs(previous_best_value-study.best_value) < self.threshold:
                  self.cb_list.append(frozen_trial.number)
                  #If threshold is achieved for the patience amount of time
                  if len(self.cb_list)>self.patience:
                      print('The study stops now...')
                      print('With number',frozen_trial.number ,'and value ',frozen_trial.value)
                      print('The previous and current best values are {} and {} respectively'
                              .format(previous_best_value, study.best_value))
                      study.stop()

In [ ]:
from IPython.display import clear_output
import sys

os.makedirs("models",exist_ok=True)

def objective(trial:optuna.Trial):
  #Trial will suggest a set of hyperparamters from the specified range

  # Optional to optimize larger set of parameters
  # hyperparameters = sample_ddpg_params_all(trial)

  # Optimize buffer size, batch size, learning rate
  hyperparameters = sample_ddpg_params(trial)
  #print(f'Hyperparameters from objective: {hyperparameters.keys()}')
  policy_kwargs = None  # default
  if 'policy_kwargs' in hyperparameters.keys():
    policy_kwargs = hyperparameters['policy_kwargs']
    del hyperparameters['policy_kwargs']
    #print(f'Policy keyword arguments {policy_kwargs}')
  model_ddpg = agent.get_model("ddpg",
                               policy_kwargs = policy_kwargs,
                               model_kwargs = hyperparameters )
  #You can increase it for better comparison
  trained_ddpg = agent.train_model(model=model_ddpg,
                                   tb_log_name="ddpg",
                                   total_timesteps=total_timesteps)
  trained_ddpg.save('models/ddpg_{}.pth'.format(trial.number))
  clear_output(wait=True)
  #For the given hyperparamters, determine the account value in the trading period
  df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg,
    environment = e_trade_gym)

  # Calculate trade performance metric
  # Currently ratio of average win and loss market values
  tpm = calc_trade_perf_metric(df_actions,trade,tp_metric)
  return tpm

#Create a study object and specify the direction as 'maximize'
#As you want to maximize sharpe
#Pruner stops not promising iterations
#Use a pruner, else you will get error related to divergence of model
#You can also use Multivariate samplere
#sampler = optuna.samplers.TPESampler(multivarite=True,seed=42)
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(study_name="ddpg_study",direction='maximize',
                            sampler = sampler, pruner=optuna.pruners.HyperbandPruner())

logging_callback = LoggingCallback(threshold=lc_threshold,
                                   patience=lc_patience,
                                   trial_number=lc_trial_number)
#You can increase the n_trials for a better search space scanning
study.optimize(objective, n_trials=1,catch=(ValueError,),callbacks=[logging_callback])

<ipython-input-21-a2d7bc23661b>:34: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  idx = pd.to_datetime(df_actions.index, infer_datetime_format=True)
[I 2025-06-08 14:41:22,297] Trial 0 finished with value: 7.1360964872380475 and parameters: {'buffer_size': 1000000, 'learning_rate': 0.0014117777290860705, 'batch_size': 64}. Best is trial 0 with value: 7.1360964872380475.


hit end!
Number of tickers: 28
Tickers: ['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV'
 'UNH' 'VZ' 'WBA' 'WMT']
Calculating profit/loss for each ticker
Ratio Avg Win/Avg Loss: 7.1360964872380475


In [ ]:
joblib.dump(study, "final_ddpg_study__.pkl")

['final_ddpg_study__.pkl']

In [ ]:
#Get the best hyperparamters
print('Hyperparameters after tuning',study.best_params)
print('Hyperparameters before tuning',config.DDPG_PARAMS)

Hyperparameters after tuning {'buffer_size': 1000000, 'learning_rate': 0.0014117777290860705, 'batch_size': 64}
Hyperparameters before tuning {'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}


In [ ]:
study.best_trial

FrozenTrial(number=0, state=1, values=[7.1360964872380475], datetime_start=datetime.datetime(2025, 6, 8, 14, 40, 52, 720374), datetime_complete=datetime.datetime(2025, 6, 8, 14, 41, 22, 297521), params={'buffer_size': 1000000, 'learning_rate': 0.0014117777290860705, 'batch_size': 64}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'buffer_size': CategoricalDistribution(choices=(10000, 100000, 1000000)), 'learning_rate': FloatDistribution(high=1.0, log=True, low=1e-05, step=None), 'batch_size': CategoricalDistribution(choices=(32, 64, 128, 256, 512))}, trial_id=0, value=None)

In [ ]:
from stable_baselines3 import DDPG
tuned_model_ddpg = DDPG.load('models/ddpg_{}.pth'.format(study.best_trial.number),env=env_train)

In [ ]:
#Trading period account value with tuned model
df_account_value_tuned, df_actions_tuned = DRLAgent.DRL_prediction(
    model=tuned_model_ddpg,
    environment = e_trade_gym)

hit end!


In [ ]:
def add_trade_perf_metric(df_actions,
                          perf_stats_all,
                          trade,
                          tp_metric):
  tpm = calc_trade_perf_metric(df_actions,trade,tp_metric)
  trp_metric = {'Value':tpm}
  df2 = pd.DataFrame(trp_metric,index=['Trade_Perf'])
  perf_stats_all = perf_stats_all.append(df2)
  return perf_stats_all

In [ ]:
#Now train with not tuned hyperaparameters
#Default config.ddpg_PARAMS
non_tuned_model_ddpg = agent.get_model("ddpg",model_kwargs = config.DDPG_PARAMS )
trained_ddpg = agent.train_model(model=non_tuned_model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=total_timesteps)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'roll

In [ ]:
trained_models_ddpg = []

for i, agent in enumerate(agent_list):
    print(f"\n🔁 Entrenando agente {i+1}/{num_agents}...")

    model = agent.get_model("ddpg", model_kwargs=config.DDPG_PARAMS)

    trained_model = agent.train_model(
        model=model,
        tb_log_name=f'ddpg_agent_{i}',
        total_timesteps=total_timesteps
    )

    trained_models_ddpg.append(trained_model)

Se han truncado las últimas 5000 líneas del flujo de salida.
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Err

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg,
    environment = e_trade_gym)

hit end!


In [ ]:
#Backtesting for not tuned hyperparamters
print("==============Get Backtest Results===========")
print("============Default Hyperparameters==========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.columns = ['Value']
# add trade performance metric
perf_stats_all
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
============Default Hyperparameters==========
Annual return          0.301444
Cumulative returns     0.453975
Annual volatility      0.174630
Sharpe ratio           1.601392
Calmar ratio           3.006932
Stability              0.949786
Max drawdown          -0.100250
Omega ratio            1.325964
Sortino ratio          2.285551
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.093042
Daily value at risk   -0.020892
dtype: float64


In [ ]:
#Hay que comparar todos contra este.
market_sharpe = perf_stats_all["Value"]["Sharpe ratio"]

In [ ]:
import pandas as pd
import numpy as np
import datetime

# Almacenar todos los sharpes
sharpes = []

for i in range(num_agents):
    print(f"📈 Evaluando agente {i+1}/{num_agents}")
    df_account_value, _ = DRLAgent.DRL_prediction(
        model=trained_models_ddpg[i],
        environment=e_trade_gym_list[i]
    )

    perf_stats = backtest_stats(account_value=df_account_value)
    sharpe = perf_stats.get('Sharpe ratio', np.nan)
    sharpes.append(sharpe)

# Convertir a np.array para facilitar el reshape
sharpes_array = np.array(sharpes).reshape(25, 10)  # 25 experimentos, 10 agentes cada uno

# Calcular métricas por experimento
experiment_metrics = []

for i in range(25):
    s = sharpes_array[i]
    mean_s = np.nanmean(s)
    q3_s = np.nanquantile(s, 0.75)
    max_s = np.nanmax(s)
    experiment_metrics.append({
        "Experiment": i + 1,
        "Mean_Sharpe": mean_s,
        "Q3_Sharpe": q3_s,
        "Max_Sharpe": max_s
    })

# Guardar como DataFrame
summary_df = pd.DataFrame(experiment_metrics)
summary_df.to_csv("summary_sharpe_by_experiment.csv", index=False)

print(summary_df)


📈 Evaluando agente 1/250
hit end!
Annual return          0.185151
Cumulative returns     0.272933
Annual volatility      0.154714
Sharpe ratio           1.178762
Calmar ratio           2.445342
Stability              0.942274
Max drawdown          -0.075716
Omega ratio            1.227138
Sortino ratio          1.706322
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.010807
Daily value at risk   -0.018768
dtype: float64
📈 Evaluando agente 2/250
hit end!
Annual return          0.297114
Cumulative returns     0.447108
Annual volatility      0.193817
Sharpe ratio           1.443745
Calmar ratio           2.648324
Stability              0.921721
Max drawdown          -0.112190
Omega ratio            1.290207
Sortino ratio          2.076613
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.057714
Daily value at risk   -0.023308
dtype: float64
📈 Evaluando agente 3/250
hit end!
Annual return          0.418686
Cumu

In [ ]:
#Certainly you can afford more number of trials for further optimization
from optuna.visualization import plot_optimization_history
fig = plot_optimization_history(study)
#"./"+config.RESULTS_DIR+
fig.write_image("./"+config.RESULTS_DIR+"/opt_hist.png")
fig.show()

/usr/local/lib/python3.11/dist-packages/kaleido/scopes/base.py:188: DeprecationWarning:

setDaemon() is deprecated, set the daemon attribute instead



In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
#Hyperparamters importance

try:
  fig = plot_param_importances(study)
  fig.write_image("./"+config.RESULTS_DIR+"/params_importances.png")
  fig.show()
except:
  print('Cannot calculate hyperparameter importances: no variation')

Cannot calculate hyperparameter importances: no variation


In [ ]:
fig = plot_edf(study)
fig.write_image("./"+config.RESULTS_DIR+"/emp_dist_func.png")
fig.show()

In [ ]:
files.download('/content/final_ddpg_study__.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [ ]:
from scipy.stats import ttest_ind
import numpy as np

# Baseline Sharpe ratio (no domain randomization)
baseline_sharpe = market_sharpe  # should be a float

# DR Sharpe ratios (25 experiments × 10 agents)
sharpes_array = np.array(sharpes).reshape(25, 10)

# Metrics per experiment
dr_max = np.nanmax(sharpes_array, axis=1)
dr_q3 = np.nanquantile(sharpes_array, 0.75, axis=1)
dr_mean = np.nanmean(sharpes_array, axis=1)

# Create a matching array of baseline values
baseline_array = np.full_like(dr_mean, baseline_sharpe)

# Run t-tests
t_max = ttest_ind(baseline_array, dr_max)
t_q3 = ttest_ind(baseline_array, dr_q3)
t_mean = ttest_ind(baseline_array, dr_mean)

# Pretty print results
print("=== T-TEST RESULTS ===")
print(f"→ No DR vs Max DR:\n   t-statistic = {t_max.statistic:.10f}, p-value = {t_max.pvalue:.25f}")
print(f"→ No DR vs Q3 DR:\n   t-statistic = {t_q3.statistic:.10f}, p-value = {t_q3.pvalue:.25f}")
print(f"→ No DR vs Mean DR:\n   t-statistic = {t_mean.statistic:.10f}, p-value = {t_mean.pvalue:.25f}")


=== T-TEST RESULTS ===
→ No DR vs Max DR:
   t-statistic = -15.9533872273, p-value = 0.0000000000000000000080718
→ No DR vs Q3 DR:
   t-statistic = -14.0451187865, p-value = 0.0000000000000000012670137
→ No DR vs Mean DR:
   t-statistic = -5.2469220764, p-value = 0.0000034598711591693055741


In [1]:
import numpy as np

# Baseline Sharpe Ratio
baseline_sharpe = 1.601392

# DR metrics from your table (25 experiments)
max_sharpe = np.array([
    1.863253, 2.006231, 1.872960, 1.832878, 1.970346,
    1.913919, 2.034160, 1.911241, 1.742590, 1.726062,
    2.019963, 1.768805, 1.900887, 1.953399, 1.953691,
    1.953658, 1.873078, 1.970622, 1.949628, 1.806630,
    1.749910, 1.888595, 2.049885, 1.848652, 1.882910
])

q3_sharpe = np.array([
    1.732702, 1.855320, 1.788143, 1.746652, 1.849455,
    1.803041, 1.827360, 1.748060, 1.617830, 1.671880,
    1.764728, 1.732181, 1.730822, 1.750906, 1.759004,
    1.728486, 1.720476, 1.813374, 1.773935, 1.710974,
    1.704215, 1.729161, 1.781178, 1.770212, 1.701051
])

mean_sharpe = np.array([
    1.606896, 1.765119, 1.659716, 1.626307, 1.757641,
    1.617699, 1.630844, 1.689416, 1.540898, 1.553235,
    1.703082, 1.703302, 1.694839, 1.709777, 1.655342,
    1.693349, 1.600455, 1.756471, 1.692791, 1.601410,
    1.610169, 1.659456, 1.718803, 1.656863, 1.635116
])

def cohens_d(sample1, sample2):
    """Compute Cohen's d for two samples"""
    mean1, mean2 = np.mean(sample1), np.mean(sample2)
    std1, std2 = np.std(sample1, ddof=1), np.std(sample2, ddof=1)
    pooled_std = np.sqrt((std1**2 + std2**2) / 2)
    return (mean2 - mean1) / pooled_std

# Create arrays matching baseline mean shape
baseline_array = np.full_like(max_sharpe, baseline_sharpe)

# Compute Cohen's d
d_max = cohens_d(baseline_array, max_sharpe)
d_q3 = cohens_d(baseline_array, q3_sharpe)
d_mean = cohens_d(baseline_array, mean_sharpe)

# Print results
print(f"Cohen's d (Baseline vs max[Sharpe]): {d_max:.6f}")
print(f"Cohen's d (Baseline vs Q3[Sharpe]): {d_q3:.6f}")
print(f"Cohen's d (Baseline vs mean[Sharpe]): {d_mean:.6f}")


Cohen's d (Baseline vs max[Sharpe]): 4.577481
Cohen's d (Baseline vs Q3[Sharpe]): 3.963719
Cohen's d (Baseline vs mean[Sharpe]): 1.421017
